In [1]:
#import pydytuesday
import pandas as pd
#pydytuesday.get_date('2025-05-20')

# Option 2: Read directly from GitHub 

water_quality = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-05-20/water_quality.csv')
weather = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-05-20/weather.csv')

In [ ]:
water_quality['date'] = pd.to_datetime(water_quality['date'])
weather['date'] = pd.to_datetime(weather['date'])

# Save the DataFrames to CSV files
water_quality.to_csv('water_quality.csv', index=False)
weather.to_csv('weather.csv', index=False)
water_quality.sample(10)

,region,council,swim_site,date,time,enterococci_cfu_100ml,water_temperature_c,conductivity_ms_cm,latitude,longitude
37004,Sydney Harbour,Northern Beaches Council,Little Manly Cove,2014-10-27,10:30:00,5.0,20.0,53000.0,-33.807232,151.286808
84294,Southern Sydney,Sutherland Shire Council,North Cronulla Beach,2001-11-12,06:40:00,2.0,NaN,NaN,-34.051010,151.156590
38514,Sydney Harbour,Northern Beaches Council,Forty Baskets Pool,2014-04-29,08:31:00,2.0,20.0,53300.0,-33.803133,151.270604
107905,Northern Sydney,Northern Beaches Council,Shelly Beach (Manly),1995-12-24,06:20:00,40.0,NaN,NaN,-33.800514,151.297668
51992,Sydney Harbour,Woollahra Municipal Council,Murray Rose Pool,2010-03-02,07:57:00,11.0,NaN,55622.0,-33.871276,151.246974
6692,Northern Sydney,Northern Beaches Council,South Curl Curl Beach,2023-08-09,09:10:00,1.0,17.0,53300.0,-33.773397,151.293337
106223,Sydney City,Randwick City Council,Clovelly Beach,1996-04-10,11:30:00,76.0,NaN,NaN,-33.914578,151.267659
86883,Sydney Harbour,Northern Beaches Council,Manly Cove,2001-03-15,12:22:00,0.0,NaN,NaN,-33.799440,151.282519
98579,Sydney Harbour,Mosman Municipal Council,Clifton Gardens,1998-03-30,10:41:00,22.0,22.0,NaN,-33.839154,151.253349
63,Sydney Harbour,Northern Beaches Council,Forty Baskets Pool,2025-04-22,10:22:00,4.0,22.0,53500.0,-33.803133,151.270604


In [4]:
weather.sample(10)


,date,max_temp_C,min_temp_C,precipitation_mm,latitude,longitude
8786,2015-01-21,24.3,20.9,5.7,-33.848858,151.19551
5697,2006-08-07,14.8,8.1,0.2,-33.848858,151.19551
1461,1995-01-01,28.2,18.9,2.2,-33.848858,151.19551
9425,2016-10-21,25.6,15.8,8.6,-33.848858,151.19551
2913,1998-12-23,23.3,18.5,0.8,-33.848858,151.19551
3862,2001-07-29,16.1,13.5,0.1,-33.848858,151.19551
4410,2003-01-28,24.1,21.0,1.1,-33.848858,151.19551
8867,2015-04-12,20.7,16.2,0.2,-33.848858,151.19551
7278,2010-12-05,23.0,19.3,2.4,-33.848858,151.19551
5300,2005-07-06,17.7,9.2,0.0,-33.848858,151.19551


In [6]:
common_dates = set(weather['date']).intersection(set(water_quality['date']))
print(f'Number of common dates: {len(common_dates)}')

Number of common dates: 5175


In [7]:
# rename columns for clarity
water_quality.rename(columns={
    'swim_site': 'beach',
    'enterococci_cfu_100ml': 'enterococci',
    'water_temperature_c': 'water_temperature',
    'conductivity_ms_cm': 'conductivity'}, inplace=True)

# Which swim sites consistently have high enterococci levels?
high_enterococci_sites = water_quality[water_quality['enterococci'] > 104].groupby('beach').size().reset_index(name='count')
high_enterococci_sites = high_enterococci_sites.sort_values(by='count', ascending=False)
high_enterococci_sites.head(10)

,beach,count
4,Boat Harbour,513
18,Darling Harbour,359
37,Malabar Beach,339
19,Davidson Reserve,266
28,Greenhills Beach,258
66,Tambourine Bay,227
17,Coogee Beach,216
55,Queenscliff Beach,213
60,South Cronulla Beach,207
76,Woolwich Baths,201


In [8]:
# How does water quality vary by season or month?
water_quality['month'] = water_quality['date'].dt.month

# Function to map month to season
def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Autumn'
    elif month in [6, 7, 8]:
        return 'Winter'
    else:
        return 'Spring'

water_quality['season'] = water_quality['month'].apply(month_to_season)
seasonal_quality = water_quality.groupby(['season', 'beach']).agg({'enterococci': 'mean',
                                                                      'water_temperature': 'mean',
                                                                      'conductivity': 'mean'}).reset_index()
seasonal_quality = seasonal_quality.sort_values(by='enterococci', ascending=False)
seasonal_quality.head(10)


,season,beach,enterococci,water_temperature,conductivity
66,Autumn,Tambourine Bay,3326.583120,21.188406,42973.584416
76,Autumn,Woolwich Baths,2430.580247,21.306569,44975.732026
75,Autumn,Woodford Bay,2375.720988,21.395349,45903.777778
57,Autumn,Sangrado Baths,1173.320574,20.222222,49050.000000
151,Spring,Wentworth Falls Lake - Jetty,855.682927,18.444444,1980.250000
129,Spring,Northbridge Baths,750.720812,19.765152,51134.905660
145,Spring,Tambourine Bay,729.519582,19.819549,47416.469136
18,Autumn,Darling Harbour,665.755319,20.889764,45161.006579
35,Autumn,Little Sirius Cove,583.040000,20.305556,54505.636364
19,Autumn,Davidson Reserve,573.377551,21.266667,41097.496732


In [15]:
common_dates = water_quality[water_quality['date'].isin(weather['date'])]

In [19]:
common_dates = common_dates.merge(weather[['date', 'precipitation_mm']], on='date', how='inner')
common_dates

,region,council,beach,date,time,enterococci,water_temperature,conductivity,latitude,longitude,month,season,precipitation_mm_x,precipitation_mm_y
0,Western Sydney,Hawkesbury City Council,Windsor Beach,2025-04-28,11:00:00,620.0,20.0,248.0,-33.604483,150.817039,4,Autumn,1.8,1.8
1,Sydney Harbour,North Sydney Council,Hayes Street Beach,2025-04-28,11:40:00,64.0,21.0,45250.0,-33.841715,151.219382,4,Autumn,1.8,1.8
2,Sydney Harbour,Willoughby City Council,Northbridge Baths,2025-04-28,10:54:00,160.0,21.0,48930.0,-33.806043,151.222754,4,Autumn,1.8,1.8
3,Sydney Harbour,Northern Beaches Council,Fairlight Beach,2025-04-28,09:28:00,54.0,21.0,52700.0,-33.800731,151.274778,4,Autumn,1.8,1.8
4,Western Sydney,Hawkesbury City Council,Yarramundi Reserve,2025-04-28,10:35:00,720.0,18.0,64.0,-33.609123,150.697886,4,Autumn,1.8,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123525,Southern Sydney,Sutherland Shire Council,North Cronulla Beach,1993-01-02,NaN,8.0,NaN,NaN,-34.051010,151.156590,1,Summer,0.0,0.0
123526,Southern Sydney,Sutherland Shire Council,Greenhills Beach,1993-01-02,NaN,4.0,NaN,NaN,-34.034790,151.184180,1,Summer,0.0,0.0
123527,Southern Sydney,Sutherland Shire Council,Elouera Beach,1993-01-02,NaN,1.0,NaN,NaN,-34.046610,151.159660,1,Summer,0.0,0.0
123528,Northern Sydney,Northern Beaches Council,Queenscliff Beach,1992-09-19,NaN,0.0,NaN,NaN,-33.786890,151.288740,9,Spring,3.4,3.4


In [20]:
common_dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123530 entries, 0 to 123529
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   region              123530 non-null  object        
 1   council             123530 non-null  object        
 2   beach               123530 non-null  object        
 3   date                123530 non-null  datetime64[ns]
 4   time                112338 non-null  object        
 5   enterococci         123223 non-null  float64       
 6   water_temperature   48491 non-null   float64       
 7   conductivity        44994 non-null   float64       
 8   latitude            123530 non-null  float64       
 9   longitude           123530 non-null  float64       
 10  month               123530 non-null  int32         
 11  season              123530 non-null  object        
 12  precipitation_mm_x  123530 non-null  float64       
 13  precipitation_mm_y  123530 no

In [37]:
df = common_dates.groupby(['beach', 'month']).agg({
    'enterococci': 'mean',
    'water_temperature': 'mean',
    'conductivity': 'mean',
    'precipitation_mm_x': 'sum'
}).reset_index()
df = df.sort_values(by=['precipitation_mm_x'], ascending=False)
df.head(50)

,beach,month,enterococci,water_temperature,conductivity,precipitation_mm_x
170,Clontarf Pool,3,84.842105,22.683333,49319.069444,707.0
14,Balmoral Baths,3,118.329412,22.491525,51135.054795,707.0
596,Northbridge Baths,3,199.766082,23.135593,46045.328767,707.0
663,Rose Bay Beach,3,106.987805,22.103448,50296.287671,705.1
302,Forty Baskets Pool,3,51.522293,22.137931,51832.369863,700.3
230,Davidson Reserve,3,407.312102,22.898305,39958.410959,700.3
98,Cabarita Beach,3,200.233766,23.140496,44533.027778,694.6
146,Chiswick Baths,3,182.090909,22.961538,45851.375000,684.0
350,Greenwich Baths,3,42.482353,22.655172,48973.931507,666.5
458,Manly Cove,3,65.976471,22.100000,52272.041096,666.5


In [36]:
common_dates.groupby('month').agg({'enterococci': 'mean'}).sort_values(by='enterococci', ascending=False).head(10)

,enterococci
month,
4,361.869173
2,143.767875
3,111.182532
5,109.619627
9,108.727535
6,102.885698
1,100.507788
8,84.102897
11,81.577896


In [14]:
# filter for common dates
common_dates = water_quality[water_quality['date'].isin(weather['date'])]

# Merge water quality with weather data on date but only take precipitation
common_dates = common_dates.merge(weather[['date', 'precipitation_mm']], on='date', how='inner')

# keep only relevant columns
merged_data = common_dates[['date', 'beach', 'council', 'region', 'enterococci', 'water_temperature', 'conductivity',
                            'latitude', 'longitude', 'precipitation_mm']]

# Save the merged data to a new CSV file
merged_data.to_csv('merged_water_quality_weather.csv', index=False)

# remove rows with NaN values

merged_data = merged_data.dropna().copy()

# remove outliers
def remove_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
merged_data = remove_outliers(merged_data, 'enterococci')
merged_data = remove_outliers(merged_data, 'water_temperature')
merged_data = remove_outliers(merged_data, 'conductivity')

# Save the cleaned merged data to a new CSV file
merged_data.to_csv('cleaned_merged_water_quality_weather.csv', index=False)

# Display the first few rows of the cleaned merged data
merged_data


,date,beach,council,region,enterococci,water_temperature,conductivity,latitude,longitude,precipitation_mm
13,2025-04-28,Chinamans Beach,Mosman Municipal Council,Sydney Harbour,26.0,21.0,51500.0,-33.814094,151.248971,1.8
21,2025-04-28,Edwards Beach,Mosman Municipal Council,Sydney Harbour,20.0,22.0,52900.0,-33.821380,151.252830,1.8
28,2025-04-28,Clifton Gardens,Mosman Municipal Council,Sydney Harbour,26.0,22.0,50900.0,-33.839154,151.253349,1.8
30,2025-04-28,Nielsen Park,Woollahra Municipal Council,Sydney Harbour,24.0,22.0,53100.0,-33.849925,151.266231,1.8
31,2025-04-24,Newport Beach,Northern Beaches Council,Northern Sydney,0.0,22.0,53700.0,-33.653976,151.323750,2.2
...,...,...,...,...,...,...,...,...,...,...
58036,2008-07-08,Nielsen Park,Woollahra Municipal Council,Sydney Harbour,0.0,17.0,55400.0,-33.849925,151.266231,0.0
58077,2008-07-02,Chinamans Beach,Mosman Municipal Council,Sydney Harbour,2.0,16.0,54300.0,-33.814094,151.248971,0.0
58094,2008-07-02,Cabarita Beach,City of Canada Bay Council,Sydney Harbour,8.0,14.0,51200.0,-33.841448,151.118630,0.0
58096,2008-07-02,Manly Cove,Northern Beaches Council,Sydney Harbour,2.0,16.0,54800.0,-33.799440,151.282519,0.0


In [17]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31254 entries, 13 to 58102
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               31254 non-null  datetime64[ns]
 1   beach              31254 non-null  object        
 2   council            31254 non-null  object        
 3   region             31254 non-null  object        
 4   enterococci        31254 non-null  float64       
 5   water_temperature  31254 non-null  float64       
 6   conductivity       31254 non-null  float64       
 7   latitude           31254 non-null  float64       
 8   longitude          31254 non-null  float64       
 9   precipitation_mm   31254 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 2.6+ MB


In [ ]:
# Are there noticeable improvements or declines in water quality over the years?

merged_data['year'] = merged_data['date'].dt.year
yearly_trends = merged_data.groupby(['year', 'beach']).agg({
    'enterococci': 'mean',
}).reset_index()
yearly_trends = yearly_trends.sort_values(by=['year', 'enterococci'], ascending=[True, False])
yearly_trends.head(10)
yearly_trends.tail(10)

# Visualizing the trends
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
sns.lineplot(data=yearly_trends, x='year', y='enterococci', hue='beach')
plt.title('Yearly Trends in Enterococci Levels by Beach')
plt.xlabel('Year')
plt.ylabel('Mean Enterococci Levels (CFU/100ml)')
plt.xticks(rotation=45)
plt.legend(title='Beach', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# What is the distribution of enterococci levels across different councils?
council_data = merged_data.groupby('council').agg({'enterococci': 'mean'}).reset_index().sort_values(by='enterococci', ascending=False).head(10)
council_data
# Visualizing the distribution of enterococci levels across councils
plt.figure(figsize=(12, 6))
sns.barplot(data=council_data, x='council', y='enterococci', palette='viridis')
plt.title('Mean Enterococci Levels by Council')
plt.xlabel('Council')
plt.ylabel('Mean Enterococci Levels (CFU/100ml)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# What is the distribution of enterococci levels across different regions?
region_data = merged_data.groupby('region').agg({'enterococci': 'mean'}).reset_index().sort_values(by='enterococci', ascending=False).head(10)
# Visualizing the distribution of enterococci levels across regions
plt.figure(figsize=(12, 6))
sns.barplot(data=region_data, x='region', y='enterococci', palette='viridis')
plt.title('Mean Enterococci Levels by Region')
plt.xlabel('Region')
plt.ylabel('Mean Enterococci Levels (CFU/100ml)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:

# Does water temperature or conductivity correlate with enterococci levels?

# Explore whether warmer or more saline water impacts bacteria levels


In [ ]:
# Do certain months with high rainfall consistently have worse water quality?
merged_data['month'] = merged_data['date'].dt.month
rainfall_quality = merged_data[merged_data['precipitation_mm'] > 10].groupby('month').agg({
    'enterococci': 'mean',
    'precipitation_mm': 'sum',
    'water_temperature': 'mean',
    'conductivity': 'mean'
}).reset_index()
rainfall_quality = rainfall_quality.sort_values(by='enterococci', ascending=False)
rainfall_quality.head(10)

In [ ]:
# Are storms followed by poor swim conditions?
# Identify storms in the weather data
# Create a new column to classify days as storm based on temperature and precipitation

storm_threshold = 20  # Example threshold for storm (high precipitation)
merged_data['event'] = merged_data.apply(
    lambda row: 'Storm' if row['precipitation_mm'] > storm_threshold else 'Normal',
    axis=1
)
# Group by event type and calculate mean enterococci levels
event_quality = merged_data.groupby('event').agg({
    'enterococci': 'mean',
    'water_temperature': 'mean',
    'conductivity': 'mean'
}).reset_index()
event_quality = event_quality.sort_values(by='enterococci', ascending=False)
event_quality.head(10)

In [ ]:
merged_data.head(10)
merged_data['season'] = merged_data['month'].apply(month_to_season)
merged_data.head(10)
merged_data.info()

In [ ]:
import matplotlib.pyplot as plt
merged_data['enterococci'].hist(bins=50)
plt.title('Distribution of Enterococci')
plt.show()


In [ ]:
# Can we predict enterococci levels from weather variables?
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
# Prepare the data for modeling
merged_data['season'] = merged_data['month'].apply(month_to_season)
# Convert season to categorical codes
merged_data['season_code'] = merged_data['season'].astype('category').cat.codes
merged_data['event_code'] = merged_data['event'].astype('category').cat.codes
merged_data['region_code'] = merged_data['region'].astype('category').cat.codes
merged_data['beach_code'] = merged_data['beach'].astype('category').cat.codes



X = merged_data[['precipitation_mm',  'conductivity',  'month','latitude', 'longitude']]


# Convert categorical and datetime columns to numeric
X = X.copy()
X['month'] = X['month'].astype(int)  # Ensure month is treated as numeric
X['latitude'] = X['latitude'].astype(float)
X['longitude'] = X['longitude'].astype(float)
import numpy as np
# Handle any NaN values in the features
X = X.replace([np.inf, -np.inf], np.nan).dropna()
# Define the target variable
y = merged_data['enterococci']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create and train the model
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)
# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


In [ ]:
# Use latitude/longitude to cluster geographically
from sklearn.cluster import KMeans
# Select relevant features for clustering
features = merged_data[['latitude', 'longitude']]
# Perform KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(features)
# Add cluster labels to the DataFrame
merged_data['cluster'] = kmeans.labels_
# Display the first few rows with cluster labels
print(merged_data.head())
# Visualize the clusters on a scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(merged_data['longitude'], merged_data['latitude'], c=merged_data['cluster'], cmap='viridis', alpha=0.5)
plt.title('Geographical Clusters of Swim Sites')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(merged_data,
                        lat='latitude',
                        lon='longitude',
                        color='enterococci',  
                        color_continuous_scale='Viridis',
                        size_max=10,
                        zoom=5,
                        mapbox_style='carto-positron')

fig.show()

In [ ]:
from ipyleaflet import Map, Marker
# Create a map centered around Sydney
sydney_map = Map(center=(-33.8688, 151.2093),
                    zoom=10,
                    scroll_wheel_zoom=True)
sydney_map

In [ ]:
marker = Marker(location=(-33.8688, 151.2093), title="Sydney")
sydney_map.add_layer(marker)
